In [1]:
from selenium.webdriver.common.by import By
import pandas as pd
import unidecode
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import sys
import requests
import re

In [2]:
url = 'https://5lb.ru'

In [3]:

option = Options()
option.add_argument("--disable-infobars") 
browser = webdriver.Chrome('//Users/iskomutkov/projects/python_course_project/chromedriver',chrome_options=option)

/var/folders/6f/3vj27vmj1_g4hyzs0s9k_3cm0000gn/T/ipykernel_3014/630198229.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('//Users/iskomutkov/projects/python_course_project/chromedriver',chrome_options=option)
/var/folders/6f/3vj27vmj1_g4hyzs0s9k_3cm0000gn/T/ipykernel_3014/630198229.py:3: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('//Users/iskomutkov/projects/python_course_project/chromedriver',chrome_options=option)


In [5]:
browser.get(url)

In [6]:
sport_nutrition = browser.find_element(By.CSS_SELECTOR, "#header > div.header-wrapper.fix-logo2.header-v8 > div.menu-row.middle-block.bgdark > div > div > div > div > nav > div > table > tbody > tr > td:nth-child(3) > div > a")
sport_nutrition.click()

In [84]:
acids = browser.find_element(By.CSS_SELECTOR, "#bx_1847241719_228 > div.section-compact-list__info > a")
acids.click()

In [16]:
def remove_spaces_enters(s):
    s = s.strip()
    s = s.replace('  ', '')
    s = s.replace('\n', '')
    return s

In [85]:
#j = 0
html_acids = browser.page_source
acid_url = browser.current_url
acids_soup = BeautifulSoup(html_acids)
result = []
while True:
    #j += 1
    a = acids_soup.find_all("div", {"class": "inner_wrap TYPE_1"})
    for item in a:
        ref = url+item.find('a').attrs['href']
        acid = requests.get(ref).text#.replace('\t', '')
        #acid = acid.replace('\n', '')
        soup = BeautifulSoup(acid)
        
        name = soup.find_all("h1", {"id": "pagetitle"})
        name = name[0].text
        
        properties = soup.find_all("div", {"class": "properties list"})
        props = {'name': name}
        for key, value in zip(properties[0].find_all("span", {"class": "properties__title-text"}),
                             properties[0].find_all("div", {"class": "properties__value darken properties__item--inline"})):
            
            props[remove_spaces_enters(key.text)] = remove_spaces_enters(value.text)
        
        price = soup.find_all("span", {"class": "price_value"})
        if len(price) > 0:
            price = int(unidecode.unidecode(price[0].getText()).replace(' ',''))
        else:
            try:
                vp_start = acid.find('price_value')
                price = int(re.split('[<>]', acid[vp_start:vp_start + 40])[1].replace('&nbsp;', ''))
            except Exception:
                continue
        props['price'] = price

        characteristic = soup.find_all("table", {"class": "props_list nbg"})
        if len(characteristic) > 0:
            characteristic = characteristic[0].findAll('span')
            for i in range(len(characteristic) // 2):
                props[characteristic[i * 2].text] = characteristic[i * 2 + 1].text
            
        result.append(props)
        
    next_page = acids_soup.find_all("a", {"class": "flex-next"})
    if len(next_page) == 0:
        break
    ref = url + next_page[0].attrs['href']
    browser.get(ref)
    html_acids = browser.page_source
    acid_url = browser.current_url
    acids_soup = BeautifulSoup(html_acids)

In [88]:
acids_df = pd.DataFrame(result)
acids_df.to_csv('acids.csv')

In [14]:
def parse(browser, file_name):
    html_catalog = browser.page_source
    catalog_url = browser.current_url
    catalog_soup = BeautifulSoup(html_catalog)
    result = []
    while True:
        #j += 1
        a = catalog_soup.find_all("div", {"class": "inner_wrap TYPE_1"})
        for item in a:
            ref = url+item.find('a').attrs['href']
            page = requests.get(ref).text
            soup = BeautifulSoup(page)

            name = soup.find_all("h1", {"id": "pagetitle"})
            name = name[0].text

            properties = soup.find_all("div", {"class": "properties list"})
            props = {'name': name}
            for key, value in zip(properties[0].find_all("span", {"class": "properties__title-text"}),
                                 properties[0].find_all("div", {"class": "properties__value darken properties__item--inline"})):

                props[remove_spaces_enters(key.text)] = remove_spaces_enters(value.text)

            price = soup.find_all("span", {"class": "price_value"})
            if len(price) > 0:
                price = int(unidecode.unidecode(price[0].getText()).replace(' ',''))
            else:
                try:
                    vp_start = page.find('price_value')
                    price = int(re.split('[<>]', page[vp_start:vp_start + 40])[1].replace('&nbsp;', ''))
                except Exception:
                    continue
            props['price'] = price

            characteristic = soup.find_all("table", {"class": "props_list nbg"})
            if len(characteristic) > 0:
                characteristic = characteristic[0].findAll('span')
                for i in range(len(characteristic) // 2):
                    props[characteristic[i * 2].text] = characteristic[i * 2 + 1].text

            result.append(props)

        next_page = catalog_soup.find_all("a", {"class": "flex-next"})
        if len(next_page) == 0:
            break
        ref = url + next_page[0].attrs['href']
        browser.get(ref)
        html_catalog = browser.page_source
        catalog_url = browser.current_url
        catalog_soup = BeautifulSoup(html_catalog)
    df = pd.DataFrame(result)
    df.to_csv(f'{file_name}.csv', index=False)

In [ ]:
browser.get(url)

In [ ]:
sport_nutrition = browser.find_element(By.CSS_SELECTOR, "#header > div.header-wrapper.fix-logo2.header-v8 > div.menu-row.middle-block.bgdark > div > div > div > div > nav > div > table > tbody > tr > td:nth-child(3) > div > a")
sport_nutrition.click()

In [ ]:
proteins = browser.find_element(By.CSS_SELECTOR, "#bx_1847241719_229 > div.section-compact-list__info > a")
proteins.click()

In [ ]:
parse(browser, 'proteins')

In [9]:
browser.get(url)

In [11]:
sport_nutrition = browser.find_element(By.CSS_SELECTOR, "#header > div.header-wrapper.fix-logo2.header-v8 > div.menu-row.middle-block.bgdark > div > div > div > div > nav > div > table > tbody > tr > td:nth-child(3) > div > a")
sport_nutrition.click()

In [12]:
proteins = browser.find_element(By.CSS_SELECTOR, "#bx_1847241719_231 > div.section-compact-list__info > a")
proteins.click()

In [17]:
parse(browser, 'energy')